# Exercise 4: Circuit Decomposition
Wow! If you managed to solve the first three exercises, congratulations! The fourth problem is supposed to puzzle even the quantum experts among you, so don’t worry if you cannot solve it. If you can, hats off to you!

You may recall from your quantum mechanics course that quantum theory is unitary. Therefore, the evolution of any (closed) system can be described by a unitary. But given an arbitrary unitary, can you actually implement it on your quantum computer?

**"A set of quantum gates is said to be universal if any unitary transformation of the quantum data can be efficiently approximated arbitrarily well as a sequence of gates in the set."** (https://qiskit.org/textbook/ch-algorithms/defining-quantum-circuits.html)

Every gate you run on the IBM Quantum Experience is transpiled into single qubit rotations and CNOT (CX) gates. We know that these constitute a universal gate set, which implies that any unitary can be implemented using only these gates. However, in general it is not easy to find a good decomposition for an arbitrary unitary. Your task is to find such a decomposition.

You are given the following unitary:

In [1]:
from may4_challenge.ex4 import get_unitary
U = get_unitary()
#print(U)
print("U has shape", U.shape)
      

U has shape (16, 16)


#### What circuit would make such a complicated unitary?

Is there some symmetry, or is it random? We just updated Qiskit with the introduction of a quantum circuit library (https://github.com/Qiskit/qiskit-terra/tree/master/qiskit/circuit/library). This library gives users access to a rich set of well-studied circuit families, instances of which can be used as benchmarks (quantum volume), as building blocks in building more complex circuits (adders), or as tools to explore quantum computational advantage over classical computation (instantaneous quantum polynomial complexity circuits).

In [22]:
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister

from may4_challenge.ex4 import check_circuit, submit_circuit

**Using only single qubit rotations and CNOT gates, find a quantum circuit that approximates that unitary $U$ by a unitary $V$ up to an error $\varepsilon = 0.01$, such that $\lVert U - V\rVert_2 \leq \varepsilon$ !** 

Note that the norm we are using here is the spectral norm, $\qquad \lVert A \rVert_2 = \max_{\lVert \psi \rVert_2= 1} \lVert A \psi \rVert$.

This can be seen as the largest scaling factor that the matrix $A$ has on any initial (normalized) state $\psi$. One can show that this norm corresponds to the largest singular value of $A$, i.e., the square root of the largest eigenvalue of the matrix $A^\dagger A$, where $A^{\dagger}$ denotes the conjugate transpose of $A$.

**When you submit a circuit, we remove the global phase of the corresponding unitary $V$ before comparing it with $U$ using the spectral norm. For example, if you submit a circuit that generates $V = \text{e}^{i\theta}U$, we remove the global phase $\text{e}^{i\theta}$ from $V$ before computing the norm, and you will have a successful submission. As a result, you do not have to worry about matching the desired unitary, $U$, up to a global phase.**

As the single-qubit gates have a much higher fidelity than the two-qubit gates, we will look at the number of CNOT-gates, $n_{cx}$, and the number of u3-gates, $n_{u3}$, to determine the cost of your decomposition as 

$$
\qquad \text{cost} = 10 \cdot n_{cx} + n_{u3}
$$

Try to optimize the cost of your decomposition. 

**Note that you will need to ensure that your circuit is composed only of $u3$ and $cx$ gates. The exercise is considered correctly solved if your cost is smaller than 1600.**

---
For useful tips to complete this exercise as well as pointers for communicating with other participants and asking questions, please take a look at the following [repository](https://github.com/qiskit-community/may4_challenge_exercises). You will also find a copy of these exercises, so feel free to edit and experiment with these notebooks.

---

In [1]:
from may4_challenge.ex4 import get_unitary
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from may4_challenge.ex4 import check_circuit, submit_circuit
from qiskit.compiler import transpile
from qiskit import Aer
import numpy as np
import math
from qiskit import *
from qiskit.tools.visualization import *
from qiskit.visualization import *
from scipy.linalg import hadamard
from sympy import *

U = get_unitary()

q = QuantumRegister(4)
c = ClassicalRegister(4)
qc = QuantumCircuit(q,c)
# H = np.array( [
#    [1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1],
#    [1 , -1 ,  1 , -1 ,  1 , -1 ,  1 , -1 ,  1 , -1 ,  1 , -1 ,  1 , -1 ,  1 , -1],
#    [1 ,  1 , -1 , -1 ,  1 ,  1 , -1 , -1 ,  1 ,  1 , -1 , -1 ,  1 ,  1 , -1 , -1],
#    [1 , -1 , -1 ,  1 ,  1 , -1 , -1 ,  1 ,  1 , -1 , -1 ,  1 ,  1 , -1 , -1 ,  1],
#    [1 ,  1 ,  1 ,  1 , -1 , -1 , -1 , -1 ,  1 ,  1 ,  1 ,  1 , -1 , -1 , -1 , -1],
#    [1 , -1 ,  1 , -1 , -1 ,  1 , -1 ,  1 ,  1 , -1 ,  1 , -1 , -1 ,  1 , -1 ,  1],
#    [1 ,  1 , -1 , -1 , -1 , -1 ,  1 ,  1 ,  1 ,  1 , -1 , -1 , -1 , -1 ,  1 ,  1],
#    [1 , -1 , -1 ,  1 , -1 ,  1 ,  1 , -1 ,  1 , -1 , -1 ,  1 , -1 ,  1 ,  1 , -1],
#    [1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1],
#    [1 , -1 ,  1 , -1 ,  1 , -1 ,  1 , -1 , -1 ,  1 , -1 ,  1 , -1 ,  1 , -1 ,  1],
#    [1 ,  1 , -1 , -1 ,  1 ,  1 , -1 , -1 , -1 , -1 ,  1 ,  1 , -1 , -1 ,  1 ,  1],
#    [1 , -1 , -1 ,  1 ,  1 , -1 , -1 ,  1 , -1 ,  1 ,  1 , -1 , -1 ,  1 ,  1 , -1],
#    [1 ,  1 ,  1 ,  1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 ,  1 ,  1 ,  1 ,  1],
#    [1 , -1 ,  1 , -1 , -1 ,  1 , -1 ,  1 , -1 ,  1 , -1 ,  1 ,  1 , -1 ,  1 , -1],
#    [1 ,  1 , -1 , -1 , -1 , -1 ,  1 ,  1 , -1 , -1 ,  1 ,  1 ,  1 ,  1 , -1 , -1],
#    [1 , -1 , -1 ,  1 , -1 ,  1 ,  1 , -1 , -1 ,  1 ,  1 , -1 ,  1 , -1 , -1 ,  1]
# ])
# H = H/4

# x = np.diag(U).tolist()
# qc.diagonal(x, q)
# P, D = U.diagonalize()  

h = hadamard(16, dtype=complex) / 4
v = np.matmul(h,U)
i = np.matmul(v,h)
%matplotlib inline
# qc.draw(output = 'mpl')
# apply operations to your quantum circuit here

qc.h(0)
qc.h(1)
qc.h(2)
qc.h(3)

# x = np.diag(i).tolist()
# qc.diagonal(x, q)
qc.iso(i,q[:4],q[4:])
qc.h(0)
qc.h(1)
qc.h(2)
qc.h(3)
qc.draw(output = 'mpl')

sim = Aer.get_backend('qasm_simulator') # we choose the simulator as our backend
new_qc = transpile(qc,backend = sim, basis_gates=['u3', 'cx'],seed_transpiler=11, optimization_level=3)
# new_qc.draw(output='mpl')
g = new_qc.decompose()
check_circuit(g)
from may4_challenge.ex4 import submit_circuit
submit_circuit(new_qc)
# plot_state_hinton(U)
# import matplotlib as mpl
# import seaborn as sns
# import numpy as np
# def plot_matrix(mat):
#     fig1, ax1 = mpl.pyplot.subplots(figsize=(10,10))
#     M1 = np.absolute(mat)
#     sns.heatmap(M1, annot =True, fmt = '0.2f', linewidths=.5, ax=ax1)
# plot_matrix(U)

# from may4_challenge.ex4 import submit_circuit
# submit_circuit(new_qc)
# f = transpile(qc_basis, basis_gates=['u3', 'cx'], optimization_level=3)
# g = f.decompose()
# g.measure([0,1,2,3],[0,1,2,3])
# result = execute(g, backend=sim,shots = 1024).result() # we run the simulation
# c = result.get_counts()
# plot_histogram(c)
# g.draw(output = 'mpl')
# check_circuit(new_qc)

      

Circuit stats:
||U-V||_2 = 7.802192551692307e-15
(U is the reference unitary, V is yours, and the global phase has been removed from both of them).
Cost is 148

Great! Your circuit meets all the constrains.
Your score is 148. The lower, the better!
Feel free to submit your answer and remember you can re-submit a new circuit at any time!


In [15]:
##### check your quantum circuit by running the next line
check_circuit(qc)

Circuit stats:
||U-V||_2 = 1.9876050222189339
(U is the reference unitary, V is yours, and the global phase has been removed from both of them).
Unable to compute cost: double check your circuit is made of `u3` and `cx` gates only.

Something is not right with your circuit: the circuit contains something else than `u3` and `cx` gates


You can check whether your circuit is valid before submitting it with `check_circuit(qc)`. Once you have a valid solution, please submit it by running the following cell (delete the `#` before `submit_circuit`). You can re-submit at any time.


In [ ]:
# Send the circuit as the final answer, can re-submit at any time
#submit_circuit(qc) 